In [ ]:
"""
LiveKit Voice Agent - Quick Start
==================================
The simplest possible LiveKit voice agent to get you started.
Requires only OpenAI and Deepgram API keys.
"""

from dotenv import load_dotenv, find_dotenv
from livekit import agents
from livekit.agents import Agent, AgentSession, RunContext
from livekit.agents.llm import function_tool
from livekit.plugins import openai, deepgram, silero, elevenlabs
from datetime import datetime
import os

# Load the nearest .env relative to the current working directory
#env_path = find_dotenv(filename=".env", usecwd=True)
load_dotenv("/Users/ganeshkrishnan/Documents/XXXXX/.env", override=True)

#print("Loaded .env from ->", env_path)

# Sanity check
print("GUIDE_FILE ->", os.getenv("GUIDE_FILE"))
print("Exists?    ->", os.path.exists(os.getenv("GUIDE_FILE") or ""))

Loaded .env from -> 
GUIDE_FILE -> /Users/ganeshkrishnan/Documents/XXXXX/guidestruct.json
Exists?    -> False


In [ ]:
class Assistant(Agent):
    """Basic voice assistant with Airbnb booking capabilities."""

    def __init__(self):
        super().__init__(
            instructions="""You are a helpful and friendly Airbnb voice assistant.
            You can help users search for Airbnbs in different cities and book their stays.
            Keep your responses concise and natural, as if having a conversation."""
        )

async def entrypoint(ctx: agents.JobContext):
    """Entry point for the agent."""

    # Configure the voice pipeline with the essentials
    session = AgentSession(
        stt=deepgram.STT(model="nova-3"),
        llm=openai.LLM(model=os.getenv("LLM_CHOICE", "gpt-4.1-mini")),
        tts=openai.TTS(voice="echo"),
        vad=silero.VAD.load(),
    )

    # Start the session
    await session.start(
        room=ctx.room,
        agent=Assistant()
    )

    # Generate initial greeting
    await session.generate_reply(
        instructions="Greet the user warmly and ask how you can help."
    )

if __name__ == "__main__":
    # Run the agent
    agents.cli.run_app(agents.WorkerOptions(entrypoint_fnc=entrypoint))